<a href="https://colab.research.google.com/github/Raph33AI/SonicSense-AI/blob/main/SonicSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:

import requests
from bs4 import BeautifulSoup

url = "https://charts.spotify.com/viral/global/weekly/latest"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

trending_titles = []
for track_row in soup.select("table tbody tr"):
    title_element = track_row.find("span", class_="chart-table-track-name")
    if title_element:
        title = title_element.text.strip()
        trending_titles.append(title)

print("The 100 most trending titles on Spotify this week:")
for i, title in enumerate(trending_titles):
    print(f"{i+1}. {title}")


The 100 most trending titles on Spotify this week:


In [63]:
# import requests
# import base64
# import random
# import time

# # Vos identifiants d'application
# CLIENT_ID = "dffbe0f471154691b105df32348a2378"  # Remplacez par votre CLIENT_ID
# CLIENT_SECRET = "cd114e5ad1654451a84a1d3b9e32c7ee"  # Remplacez par votre CLIENT_SECRET

# # Fonction pour obtenir l'access token Spotify (inchangée)
# def get_access_token():
#     auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
#     auth_bytes = auth_string.encode("utf-8")
#     auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

#     token_url = "https://accounts.spotify.com/api/token"
#     token_data = {
#         "grant_type": "client_credentials"
#     }
#     token_headers = {
#         "Authorization": f"Basic {auth_base64}"
#     }

#     token_response = requests.post(token_url, data=token_data, headers=token_headers)

#     if token_response.status_code == 200:
#         return token_response.json()["access_token"]
#     else:
#         print(f"Erreur lors de la récupération du token: {token_response.status_code}")
#         return None

# # Obtenir l'access token
# ACCESS_TOKEN = get_access_token()

# if ACCESS_TOKEN:
#     headers = {
#         "Authorization": f"Bearer {ACCESS_TOKEN}"
#     }

#     genre_demandé = input("Entrez un genre musical : ")

#     all_tracks_data = []  # Liste pour stocker les données des titres
#     offset = 0
#     limit = 50
#     total_tracks_to_fetch = 1000

#     print("Collecte des données...")

#     while len(all_tracks_data) < total_tracks_to_fetch:
#         search_url = "https://api.spotify.com/v1/search"
#         search_params = {
#             "q": f"genre:{genre_demandé}",
#             "type": "track",
#             "limit": limit,
#             "offset": offset
#         }
#         search_response = requests.get(search_url, headers=headers, params=search_params)

#         if search_response.status_code == 200:
#             search_data = search_response.json()
#             tracks = search_data["tracks"]["items"]

#             for track in tracks:
#                 artists = ", ".join([artist["name"] for artist in track["artists"]])
#                 album = track["album"]["name"]
#                 release_date = track["album"]["release_date"]
#                 track_id = track['id']

#                 audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
#                 audio_features_response = requests.get(audio_features_url, headers=headers)

#                 if audio_features_response.status_code == 200:
#                     audio_features = audio_features_response.json()
#                     tempo = audio_features.get("tempo", "N/A")
#                     danceability = audio_features.get("danceability", "N/A")
#                     energy = audio_features.get("energy", "N/A")
#                     valence = audio_features.get("valence", "N/A")

#                     # Ajouter les données du titre à la matrice
#                     all_tracks_data.append([
#                         artists, track["name"], album, release_date,
#                         tempo, danceability, energy, valence
#                     ])

#             offset += limit
#             time.sleep(0.2)

#             if len(all_tracks_data) % 100 == 0:
#                 print(f"{len(all_tracks_data)} titres collectés...")
#         else:
#             print(f"Erreur lors de la recherche de morceaux : {search_response.status_code}")
#             break

#     print(f"{len(all_tracks_data)} titres collectés au total.")

#     # Afficher la matrice
#     print("Matrice des données des titres:")
#     for row in all_tracks_data:
#         print(row)
# else:
#     print("Erreur d'authentification Spotify.")



import requests
import base64
import random
import time
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Vos identifiants d'application
CLIENT_ID = "dffbe0f471154691b105df32348a2378"  # Remplacez par votre CLIENT_ID
CLIENT_SECRET = "cd114e5ad1654451a84a1d3b9e32c7ee"  # Remplacez par votre CLIENT_SECRET

# Fonction pour obtenir l'access token Spotify
def get_access_token():
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    token_url = "https://accounts.spotify.com/api/token"
    token_data = {
        "grant_type": "client_credentials"
    }
    token_headers = {
        "Authorization": f"Basic {auth_base64}"
    }

    token_response = requests.post(token_url, data=token_data, headers=token_headers)

    if token_response.status_code == 200:
        return token_response.json()["access_token"]
    else:
        print(f"Erreur lors de la récupération du token: {token_response.status_code}")
        return None

# Obtenir l'access token
ACCESS_TOKEN = get_access_token()

if ACCESS_TOKEN:
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }

    genre_demandé = input("Entrez un genre musical : ")

    all_tracks_data = []
    offset = 0
    limit = 50
    total_tracks_to_fetch = 1000

    print("Collecte des données...")

    while len(all_tracks_data) < total_tracks_to_fetch:
        search_url = "https://api.spotify.com/v1/search"
        search_params = {
            "q": f"genre:{genre_demandé}",
            "type": "track",
            "limit": limit,
            "offset": offset
        }
        search_response = requests.get(search_url, headers=headers, params=search_params)

        if search_response.status_code == 200:
            search_data = search_response.json()
            tracks = search_data["tracks"]["items"]

            for track in tracks:
                artists = ", ".join([artist["name"] for artist in track["artists"]])
                album = track["album"]["name"]
                release_date = track["album"]["release_date"]
                track_id = track['id']

                audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
                audio_features_response = requests.get(audio_features_url, headers=headers)

                if audio_features_response.status_code == 200:
                    audio_features = audio_features_response.json()
                    tempo = float(audio_features.get("tempo", 0.0))  # Convertir en float, gérer les valeurs manquantes
                    danceability = float(audio_features.get("danceability", 0.0))
                    energy = float(audio_features.get("energy", 0.0))
                    valence = float(audio_features.get("valence", 0.0))

                    all_tracks_data.append([
                        artists, track["name"], album, release_date,
                        tempo, danceability, energy, valence
                    ])

            offset += limit
            time.sleep(0.2)

            if len(all_tracks_data) % 100 == 0:
                print(f"{len(all_tracks_data)} titres collectés...")
        else:
            print(f"Erreur lors de la recherche de morceaux : {search_response.status_code}")
            break  # <- Indentation corrigée

    print(f"{len(all_tracks_data)} titres collectés au total.")

    # --- Préparation des données pour l'entraînement ---

    # Convertir les données en tenseur PyTorch
    data = torch.tensor(all_tracks_data)
    X = data[:, 4:].float() # Sélectionner les caractéristiques numériques
    y = data[:, 4].float() # Sélectionner le tempo comme cible

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normaliser les données
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # --- Création et entraînement du modèle ---
    class RegressionModel(nn.Module):
        def __init__(self, input_size, output_size):
            super(RegressionModel, self).__init__()
            self.linear = nn.Linear(input_size, output_size)

        def forward(self, x):
            return self.linear(x)

    model = RegressionModel(input_size=4, output_size=1)

    # --- Entraînement du modèle ---
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    epochs = 100
    for epoch in range(epochs):
        # Entraînement
        model.train()
        optimizer.zero_grad()
        outputs = model(torch.from_numpy(X_train).float())
        loss = criterion(outputs, y_train.float().unsqueeze(1))  # Enlever la conversion en NumPy
        loss.backward()
        optimizer.step()

        # Afficher la perte toutes les 10 époques
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    # --- Évaluation du modèle ---
    model.eval()
    with torch.no_grad():
        y_pred = model(torch.from_numpy(X_test).float())
        test_loss = criterion(y_pred, y_test.float().unsqueeze(1))
        print(f'Test Loss: {test_loss.item():.4f}')

else:
    print("Erreur d'authentification Spotify.")


Entrez un genre musical : rap
Collecte des données...
0 titres collectés...
0 titres collectés...
0 titres collectés...
Erreur lors de la recherche de morceaux : 429
0 titres collectés au total.
Matrice des données des titres:
